<a href="https://colab.research.google.com/github/amirarasteh1990/Examples/blob/main/Topfarm_Edwin_example_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## INSTALL ALL REQUIRED PACKAGES
import importlib
if not importlib.util.find_spec("py_wake"):
  !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/PyWake.git
if not importlib.util.find_spec("topfarm"):
  !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/TopFarm2.git
if not importlib.util.find_spec("ed_win"):
  !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/edwin.git#egg=ed_win[interarray]
  # fix ortools version
  !pip uninstall ortools -y
  !pip install ortools==9.6.2534

  Cloning https://gitlab.windenergy.dtu.dk/TOPFARM/PyWake.git to /tmp/pip-req-build-m4eou0wk
  Running command git clone --filter=blob:none --quiet https://gitlab.windenergy.dtu.dk/TOPFARM/PyWake.git /tmp/pip-req-build-m4eou0wk
  Resolved https://gitlab.windenergy.dtu.dk/TOPFARM/PyWake.git to commit 18d9a666858a205641c461386f31ae558cb5b155
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.9 MB/s eta 0:00:00
  Created wheel for py_wake: filename=py_wake-2.6.9-py3-none-any.whl size=19890101 sha256=4591ded51386812acf8684f8de897db51ca85bc319d5e32950b4b5c320b2fa7e
  Stored in directory: /tmp/pip-ephem-wheel-cache-j7nz1iop/wheels/76/2f/75/c056f102fb42126f415398d5817b64ad9edbe27227039b02b3
Successfully built py_wake
  Cloning https://gitlab.windenergy.dtu.dk/TOPFARM/TopFarm2.g

In [2]:
## IMPORT ALL REQUIRED FUNCTIONS

import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
from openmdao.api import n2

from topfarm._topfarm import TopFarmProblem, TopFarmGroup
from topfarm.cost_models.cost_model_wrappers import CostModelComponent
from topfarm.plotting import XYPlotComp
from topfarm.cost_models.py_wake_wrapper import PyWakeAEPCostModelComponent
from topfarm import SpacingConstraint, XYBoundaryConstraint
from topfarm.cost_models.economic_models.dtu_wind_cm_main import economic_evaluation

from py_wake.examples.data.hornsrev1 import wt_x, wt_y, HornsrevV80, Hornsrev1Site
from py_wake import NOJ
from py_wake.utils.gradients import autograd

from ed_win.wind_farm_network import WindFarmNetwork

ImportError: cannot import name 'SpacingConstraint' from 'topfarm' (/usr/local/lib/python3.11/dist-packages/topfarm/__init__.py)

In [ ]:
## SETUP OF WIND FARM MODEL

x_init = np.array(wt_x)[np.array([0, 1, 8, 9])]
y_init = np.array(wt_y)[np.array([0, 1, 8, 9])]
n_wt = len(x_init)
wt = HornsrevV80()
site = Hornsrev1Site()
wf_model = NOJ(site, wt)
aep = wf_model(wt_x, wt_y).aep()

In [ ]:
## SETUP OF ELECTRICAL COLLECTION SYSTEM MODEL

turbines_pos=np.asarray([wt_x, wt_y]).T
substations_pos = np.asarray([[359470], [6153890]]).T

cables = np.array([[500, 3, 100], [800, 5, 150], [1000, 10, 250]])

wfn = WindFarmNetwork(turbines_pos=turbines_pos, substations_pos=substations_pos, cables=cables)

In [ ]:
## SETUP OF ECONOMICAL MODEL

Drotor_vector = [wt.diameter()] * n_wt
power_rated_vector = [float(wt.power(20))*1e-6] * n_wt
hub_height_vector = [wt.hub_height()] * n_wt

# add additional cost model inputs for shore distance, energy price, project lifetime, rated rotor speed and water depth
distance_from_shore = 30         # [km]
energy_price = 0.4               # [Euro/kWh] What we get per kWh
project_duration = 20            # [years]
rated_rpm_array = [12] * n_wt    # [rpm]
# water_depth_array = [15] * n_wt  # [m]
eco_eval = economic_evaluation(distance_from_shore, energy_price, project_duration)

In [ ]:
## SETUP OF WRAPPER FUNCTIONS

def waterdepth_func(x, y, **kwargs):
  # insert a good water depth model here
  return 20

def cable_func(x, y, x_substation, y_substation, **kwargs):
  G = wfn.optimize(turbines_pos= np.asarray([x, y]).T)
  return G.size(weight="cost")

def npv_func(AEP, water_depth, cabling_cost, **kwargs):
    eco_eval.calculate_npv(rated_rpm_array, Drotor_vector, power_rated_vector, hub_height_vector, water_depth, AEP/n_wt * np.ones(n_wt)*10**6, cabling_cost=cabling_cost)
    CAPEX = eco_eval.project_costs_sums["CAPEX"]
    OPEX = eco_eval.project_costs_sums["OPEX"]
    return [eco_eval.NPV, CAPEX, OPEX]

In [ ]:
## SETUP OF COMPONENTS

aep_component = PyWakeAEPCostModelComponent(wf_model, n_wt, grad_method=autograd, objective=False)

water_depth_component = CostModelComponent(input_keys=[('x', x_init),('y', y_init)],
                                          n_wt=n_wt,
                                          cost_function=waterdepth_func,
                                          objective=False,
                                          output_keys=[('water_depth', np.zeros(n_wt))])

cable_component = CostModelComponent(input_keys=[('x', x_init),('y', y_init), ('x_substation', 424700), ('y_substation', 6150800)],
                                     n_wt=n_wt,
                                     cost_function=cable_func,
                                     objective=False,
                                     output_keys=[('cabling_cost', 1000)])

npv_comp = CostModelComponent(input_keys=[('AEP', 0), ('water_depth', 30*np.ones(n_wt)), ('cabling_cost', 1000)],
                              n_wt=n_wt,
                              cost_function=npv_func,
                              objective=True,
                              maximize=True,
                              output_keys=[('npv', 0), ('CAPEX', 0), ('OPEX', 0)])

cost_comp = TopFarmGroup([aep_component, water_depth_component, cable_component, npv_comp])

In [ ]:
## SETUP OF PROBLEM

problem = TopFarmProblem(design_vars={'x': x_init, 'y': y_init, 'x_substation': 424700, 'y_substation': 6150800},
                  cost_comp=cost_comp,
                  constraints=[XYBoundaryConstraint(np.asarray([x_init, y_init]).T, boundary_type='rectangle'),
                              SpacingConstraint(4 * wt.diameter())],
                  plot_comp=XYPlotComp())

In [ ]:
problem.evaluate()

In [ ]:
cost, state, recorder = problem.optimize()

In [ ]:
## PLOT N2 DIAGRAM TO INSPECT YOUR PROBLEM SETUP

n2(problem, display_in_notebook=True)

In [ ]:
## PLOT WIND FARM

plt.plot(x_init, y_init, '2k', markersize=30)

In [ ]:
## ACCESSING DATA

fig = plt.figure()
plt.plot(recorder['timestamp']-recorder['timestamp'][0], recorder['npv'])
plt.xlabel("time [s]")
plt.ylabel("npv [EUR]")


In [ ]:
problem['CAPEX']

In [ ]:
problem["OPEX"]

In [ ]:
turbines_pos=np.asarray([problem['x'], problem['y']]).T
substations_pos = np.asarray([problem['x_substation'], problem['y_substation']]).T
G = wfn.optimize(turbines_pos=turbines_pos, substations_pos=substations_pos)

In [ ]:
G.plot()